## Vessel prediction of load and discharge
In this notebook, vessel data is used to forecast total load and discharge. Data containing dates and actual load and discharge on these dates has been used to make models learn to forecast these two features. The assumption is that these models learn a certain trend over time and pick up distinct ques in the data that discharge or load might be higher or lower on specific days.

A total of two models have been used, the Random Forest Regressor and the XGBoost Regressor. 
A total of four models were trained. Two of each algorithms where each algorithm was trained on load and discharge seperately.

In the end both models are compared using their MSE (mean squared error), MAE (mean absolute error) and to some extent MAPE (mean absolute percentage error). These are error functions that are often used in regression problems since it takes the distance between the prediction and actual target into account and not only whether the prediction matches the target exactly, so it is more lenient than an error function that requires the prediction to be exactly the same as the target.

This work is done by Samar Hashemi
samar-hashemi@outlook.com

In [1]:
import math
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tabulate import tabulate
import calendar

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from datetime import datetime

sns.set_theme(style="darkgrid")
!pip install openpyxl

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Data
Data Loading

In [2]:
df = pd.read_excel('VesselData.xlsx')
df.head()

,eta,ata,atd,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,...,load4,stevedorenames,hasnohamis,earliesteta,latesteta,traveltype,previousportid,nextportid,isremarkable,vesselid
0,2017-09-19 00:00:00+00,2017-09-19 00:00:00+00,2017-09-22 00:00:00+00,109290.0,5.0,0.0,0.0,0.0,0.0,90173.0,...,0.0,Stevedore_104,NaN,2017-09-19 00:00:00+00,2017-09-19 00:00:00+00,ARRIVAL,981.0,731.0,f,2242.0
1,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-03 00:00:00+00,67170.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_109,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,19.0,15.0,f,5462.0
2,2017-09-30 00:00:00+00,2017-09-30 00:00:00+00,2017-10-01 00:00:00+00,67737.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_57,NaN,2017-09-30 00:00:00+00,2017-09-30 00:00:00+00,ARRIVAL,19.0,19.0,f,5251.0
3,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-03 00:00:00+00,43600.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_57,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,15.0,18.0,f,5268.0
4,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,9231.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_98,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,74.0,27.0,f,5504.0


In [3]:
# Only keep dates and remove time
for i in range(len(df)):
    df['eta'][i] = df['eta'][i][:10]
    df['ata'][i] = df['ata'][i][:10]
    df['atd'][i] = df['atd'][i][:10]
    


/var/folders/5c/m7_27vg11j3by39pxjtnx4880000gn/T/ipykernel_83982/2222362820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eta'][i] = df['eta'][i][:10]
/var/folders/5c/m7_27vg11j3by39pxjtnx4880000gn/T/ipykernel_83982/2222362820.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ata'][i] = df['ata'][i][:10]
/var/folders/5c/m7_27vg11j3by39pxjtnx4880000gn/T/ipykernel_83982/2222362820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [4]:
# year = df['eta'][0][:4]
# month = df['eta'][0][5:7]
# day = df['eta'][0][8:]
from datetime import datetime

week_eta = []
week_ata = []
week_atd = []
day_eta = []
day_ata = []
day_atd = []
month_eta = []
month_ata = []
month_atd = []
year_ata = []
year_eta = []
year_atd = []
for i in range(len(df)):
    
    week_eta.append(calendar.day_name[(datetime.strptime(df['eta'][i], '%Y-%m-%d').date()).weekday()])
    week_ata.append(calendar.day_name[(datetime.strptime(df['ata'][i], '%Y-%m-%d').date()).weekday()])
    week_atd.append(calendar.day_name[(datetime.strptime(df['atd'][i], '%Y-%m-%d').date()).weekday()])
    
    day_eta.append(int(df['eta'][i][8:]))
    day_ata.append(int(df['ata'][i][8:]))
    day_atd.append(int(df['atd'][i][8:]))
    
    year_eta.append(int(df['eta'][i][:4]))
    year_ata.append(int(df['ata'][i][:4]))
    year_atd.append(int(df['atd'][i][:4]))
    
    month_eta.append(int(df['eta'][i][5:7]))
    month_ata.append(int(df['eta'][i][5:7]))
    month_atd.append(int(df['eta'][i][5:7]))

In [5]:
df['eta_year'] = year_eta
df['ata_year'] = year_ata
df['atd_year'] = year_atd

df['month_eta'] = month_eta
df['month_ata'] = month_ata
df['month_atd'] = month_atd

df['day_eta'] = day_eta
df['day_ata'] = day_ata
df['day_atd'] = day_atd

df['week_eta'] = week_eta
df['week_ata'] = week_ata
df['week_atd'] = week_atd

In [6]:
df.head()

,eta,ata,atd,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,...,atd_year,month_eta,month_ata,month_atd,day_eta,day_ata,day_atd,week_eta,week_ata,week_atd
0,2017-09-19,2017-09-19,2017-09-22,109290.0,5.0,0.0,0.0,0.0,0.0,90173.0,...,2017,9,9,9,19,19,22,Tuesday,Tuesday,Friday
1,2017-10-02,2017-10-02,2017-10-03,67170.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2017,10,10,10,2,2,3,Monday,Monday,Tuesday
2,2017-09-30,2017-09-30,2017-10-01,67737.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2017,9,9,9,30,30,1,Saturday,Saturday,Sunday
3,2017-10-02,2017-10-02,2017-10-03,43600.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2017,10,10,10,2,2,3,Monday,Monday,Tuesday
4,2017-10-02,2017-10-02,2017-10-02,9231.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2017,10,10,10,2,2,2,Monday,Monday,Monday


In [7]:
# Dropping features deemed irrelevant
df = df.drop('eta', axis=1)
df = df.drop('ata', axis=1)
df = df.drop('atd', axis=1)
df = df.drop('hasnohamis',axis=1)
df = df.drop('latesteta', axis=1)
df = df.drop('earliesteta', axis=1)
df = df.drop('stevedorenames',axis=1)

In [8]:
# Label creation
total_discharge = []
total_load = []

for i in range(len(df)):
    tot_d = 0
    tot_l = 0
    tot_d = int(df['discharge1'][i])+int(df['discharge2'][i])+int(df['discharge3'][i])+int(df['discharge4'][i])
    total_discharge.append(tot_d)
    tot_l = df['load1'][i]+df['load2'][i]+df['load3'][i]+df['load4'][i]
    total_load.append(tot_l)

In [9]:
df['discharge_target'] = total_discharge
df['load_target'] = total_load

### Data analysis

In [10]:
train_columns = list(df.columns)

In [11]:
# Checked each column for nan values
print('Check data for NaN values\n')
print(train_columns)

null_columns = []
for subframe in train_columns:
    print(df[subframe].isnull().values.any())
    if df[subframe].isnull().values.any() == True:
        null_columns.append(subframe)

Check data for NaN values

['vesseldwt', 'vesseltype', 'discharge1', 'load1', 'discharge2', 'load2', 'discharge3', 'load3', 'discharge4', 'load4', 'traveltype', 'previousportid', 'nextportid', 'isremarkable', 'vesselid', 'eta_year', 'ata_year', 'atd_year', 'month_eta', 'month_ata', 'month_atd', 'day_eta', 'day_ata', 'day_atd', 'week_eta', 'week_ata', 'week_atd', 'discharge_target', 'load_target']
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [12]:
#Removing rows containing null values
print(len(df))
df = df.dropna()
print(len(df))

8208
8206


In [13]:
df.head()

,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,load3,discharge4,load4,...,month_ata,month_atd,day_eta,day_ata,day_atd,week_eta,week_ata,week_atd,discharge_target,load_target
0,109290.0,5.0,0.0,0.0,0.0,0.0,90173.0,0.0,0.0,0.0,...,9,9,19,19,22,Tuesday,Tuesday,Friday,90173,0.0
1,67170.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10,10,2,2,3,Monday,Monday,Tuesday,0,0.0
2,67737.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9,9,30,30,1,Saturday,Saturday,Sunday,0,0.0
3,43600.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10,10,2,2,3,Monday,Monday,Tuesday,0,0.0
4,9231.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10,10,2,2,2,Monday,Monday,Monday,0,0.0


No NaN values present in any of the (sub)dataframes

In [14]:
df.describe()

,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,load3,discharge4,load4,...,ata_year,atd_year,month_eta,month_ata,month_atd,day_eta,day_ata,day_atd,discharge_target,load_target
count,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,...,8206.0,8206.0,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000,8206.000000
mean,37929.072630,3.593712,1733.135145,60.769193,1168.983061,19.392274,4792.499634,44.756398,1815.638923,1509.414575,...,2017.0,2017.0,7.684865,7.684865,7.684865,15.217158,15.146234,14.859006,9510.256763,1634.332440
std,51742.798795,0.987764,16299.958423,1325.625139,11332.501528,665.414263,25369.016803,3072.092242,10024.361167,11937.951028,...,0.0,0.0,1.818554,1.818554,1.818554,8.899267,8.836828,8.633579,32843.070537,12400.233806
min,624.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2017.0,2017.0,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,6600.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2017.0,2017.0,6.000000,6.000000,6.000000,7.000000,7.000000,7.000000,0.000000,0.000000
50%,13031.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2017.0,2017.0,8.000000,8.000000,8.000000,15.000000,15.000000,15.000000,0.000000,0.000000
75%,46600.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2017.0,2017.0,9.000000,9.000000,9.000000,23.000000,22.000000,22.000000,0.000000,0.000000
max,320805.000000,5.000000,204304.000000,41761.000000,189933.000000,43639.000000,299647.000000,271251.000000,183837.000000,293449.000000,...,2017.0,2017.0,11.000000,11.000000,11.000000,31.000000,31.000000,31.000000,299647.000000,293449.000000


We don't see any weird values here and since there are no null values either the data should be usable.
However we do see that the data is imbalanced as most of the discharge_target/load_target is around 0. This would mean that some tackling for data imbalance is needed. Due to time constraint this was not done but would decrease the accuracy and reflect a model having learned better. It might have decided to overfit with the current data, but since the data in general (so also the test data) is imbalanced towards 0, the hypothesis is that the model will also perform high on the test data.

### Data Preprocessing
#### One Hot encoding

In [15]:
features = pd.get_dummies(df)
# features = df
features.head()

,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,load3,discharge4,load4,...,week_ata_Thursday,week_ata_Tuesday,week_ata_Wednesday,week_atd_Friday,week_atd_Monday,week_atd_Saturday,week_atd_Sunday,week_atd_Thursday,week_atd_Tuesday,week_atd_Wednesday
0,109290.0,5.0,0.0,0.0,0.0,0.0,90173.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,0
1,67170.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,67737.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,43600.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,9231.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


#### Label extraction

In [16]:
# Obtaining the labels/targets and removing it from the features dataset
targets = features['discharge_target']
features = features.drop('discharge_target', axis=1)
targets2 = features['load_target']
features = features.drop('load_target', axis=1)
feature_list = list(features.columns)
features = np.array(features)




#### Train/Test split

In [17]:
# Split the data into training and testing sets
# Seperate sets for training discharge/load models
train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size = 0.25, random_state = 42)
train_features2, test_features2, train_labels2, test_labels2 = train_test_split(features, targets2, test_size = 0.25, random_state = 42)

Because the test.csv does not have actual labels, there is no way to evaluate the trained model with a score with the suggested evaluation method. Therefore we use this test.csv as a seperate test file used once at the end. For training the model and validation we use train test split of 75% to 25% from the original given training data. This way we can evaluate the predicted values with the actual given values.

In [18]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (6154, 46)
Training Labels Shape: (6154,)
Testing Features Shape: (2052, 46)
Testing Labels Shape: (2052,)


## Model Implementation
### Random Forest Regressor

#### Discharge Prediction Model

In [19]:
# Instantiate model with 500 decision trees
rf = RandomForestRegressor(n_estimators = 500, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

Random Forest model is susceptible for n_estimators. Increasing the n_estimators further does not improve performance significantly. Optimally a grid search for the best amount of n_estimators and further finetuning is needed for optimal performance. This is the case for all model variations below.

In [20]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
errors_mse = math.sqrt(round(np.mean(errors), 2))
errors_mae = round(np.mean(errors), 2)
# Print out the mean absolute error (mae)
print('Mean Squared Error:', errors_mse, 'degrees.')
print('Mean Absolute Error:', errors_mae, 'degrees.')


Mean Squared Error: 11.908820260630353 degrees.
Mean Absolute Error: 141.82 degrees.


In [21]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 98.33 %.


### Load Prediction Model 2

In [22]:
# Instantiate model with 500 decision trees
rf2 = RandomForestRegressor(n_estimators = 500, random_state = 42)
# Train the model on training data
rf2.fit(train_features2, train_labels2);

In [23]:
# Use the forest's predict method on the test data
predictions2 = rf2.predict(test_features2)
# Calculate the absolute errors
errors2 = abs(predictions2 - test_labels2)
error2_mse = math.sqrt(round(np.mean(errors2), 2))
error2_mae = round(np.mean(errors2), 2)
# Print out the mean absolute error (mae)
print('Mean Squared Error:', error2_mse, 'degrees.')
print('Mean Absolute Error:', error2_mae, 'degrees.')



Mean Squared Error: 8.143095234614416 degrees.
Mean Absolute Error: 66.31 degrees.


In [24]:
# Calculate mean absolute percentage error (MAPE)
mape2 = 100 * (errors2 / test_labels2)
# Calculate and display accuracy
accuracy2 = 100 - np.mean(mape2)
print('Accuracy:', round(accuracy2, 2), '%.')

Accuracy: 94.9 %.


# XGBoost Model

We also use XGBoost to make predictions on the test data.

#### XGBoost Discharge Model

In [25]:
clf = XGBRegressor()
clf.fit(train_features,train_labels)

y_pred = clf.predict(test_features)

Fitting the XGBRegressor on the training data and obtaining predictions. In reality you would want to finetune the XGBooster more but after quick comparison with some other parameters such as the eta, I concluded that for now the default parameters were sufficient.

In [26]:
import math
# Use the forest's predict method on the test data
xgb_predictions = y_pred
# Calculate the absolute errors
xgb_errors = abs(xgb_predictions - test_labels)
xgb_mse = math.sqrt(round(np.mean(xgb_errors), 2))
xgb_mae = round(np.mean(xgb_errors), 2)
# Print out the mean absolute error (mae)
print('Mean Squared Error:', xgb_mse, 'degrees.')
print('Mean Squared Error:', xgb_mae, 'degrees.')





Mean Squared Error: 12.861959415267956 degrees.
Mean Squared Error: 165.43 degrees.


In [27]:
# Calculate mean absolute percentage error (MAPE)
mapexgb = 100 * (xgb_errors / test_labels)
# Calculate and display accuracy
accuracyxgb = 100 - np.mean(mapexgb)
print('Accuracy:', round(accuracyxgb, 2), '%.')

Accuracy: -inf %.


The results from the XGBoost algorithm are worse than that of the RandomForest Algorithm. This could be due to the fact that XGBoost requires more finetuning. It has more parameters (6) compared to random forest (3). After altering the learning rate the MSE did vary, but did not improve significantly. 

### XGBoost Series 2

In [28]:
clf2 = XGBRegressor()
clf2.fit(train_features2,train_labels2)

y_pred2 = clf.predict(test_features2)

In [29]:
xgb_predictions2 = y_pred2
xgb_errors2 = abs(xgb_predictions2 - test_labels2)
xgb_mse2 = math.sqrt(round(np.mean(xgb_errors2), 2))
xgb_mae2 = round(np.mean(xgb_errors2), 2)
# Print out the mean absolute error (mse)
print('Mean Squared Error:', xgb_mse2, 'degrees.')
print('Mean Squared Error:', xgb_mae2, 'degrees.')



Mean Squared Error: 103.36851551608932 degrees.
Mean Squared Error: 10685.05 degrees.


In [30]:
# Calculate mean absolute percentage error (MAPE)
mapexgb2 = 100 * (xgb_errors2 / test_labels2)
# Calculate and display accuracy
accuracyxgb2 = 100 - np.mean(mapexgb2)
print('Accuracy:', round(accuracyxgb2, 2), '%.')

Accuracy: -inf %.


## Model Comparison

MSE scores

In [31]:
data = [[['RF_Detachment',errors_mse],['RF_Load',error2_mse]],[['XGB_Detachment',xgb_mse],['XGB_Load',xgb_mse2]]]
col_names = ['Detachment', 'Load']
print(tabulate(data, headers=col_names, tablefmt="pipe"))

| Detachment                             | Load                             |
|:---------------------------------------|:---------------------------------|
| ['RF_Detachment', 11.908820260630353]  | ['RF_Load', 8.143095234614416]   |
| ['XGB_Detachment', 12.861959415267956] | ['XGB_Load', 103.36851551608932] |


MAE scores

In [32]:
data = [[['RF_Detachment',errors_mae],['RF_Load',error2_mae]],[['XGB_Detachment',xgb_mae],['XGB_Load',xgb_mae2]]]
col_names = ['Detachment', 'Load']
print(tabulate(data, headers=col_names, tablefmt="pipe"))

| Detachment                 | Load                   |
|:---------------------------|:-----------------------|
| ['RF_Detachment', 141.82]  | ['RF_Load', 66.31]     |
| ['XGB_Detachment', 165.43] | ['XGB_Load', 10685.05] |


MAPE scores are not compared since the XGB scores -inf so there might be something wrong there.

We conclude that the Random Forest Regressor performs better than the XGBoost Regressor which could relate to a lack of finetuning and the XGBoost having more parameters. 